# Task 1 Reconstruct the Original Meeting Transcripts
#### Student Name: Yicheng Zhang
#### Student ID: 27699641

Date: 01/06/2018

Version: 1.0

Environment: Python 3.6.5 and Jupyter notebook

Libraries used:
* bs4 (for xml parse, need to be installed before using) 
* re (for regular expression, included in Anaconda Python 3.6) 
* os (for os directory, included in Anaconda Python 3.6)

## 1.  Import libraries 

In [1]:
from bs4 import BeautifulSoup as bsoup
import re
import os

## 2. Exatract information from xml files

The first task is to parse each patent stored in the different folder. The information to be extracted includes
1. Topic files
2. Segement files
3. Word files 

We can use one single file as the example of this task.

**Define the folders' path**

In [2]:
topic_file_path = "./topics"
segements_file_path = "./segments"
words_file_path = "./words"

### *The Following code may take about 1 min- 2 min (verify with the hardware), Take about 50M memory. It is very helpful if you release your computer resources*

## 3.Using the beautiful soup to discover xml information

### Explore topic file

In [3]:
xmlSoup1 = bsoup(open('./topics/ES2002a.topic.xml'),"lxml") # Extract information using lxml

In [4]:
xmlSoup1.topic.findParent()  # find parent tags

<nite:root nite:id="ES2002a.topic" xmlns:nite="http://nite.sourceforge.net/">
<topic nite:id="ES2002a.topic.vkaraisk.1" other_description="introduction of participants and their roles">
<nite:pointer href="default-topics.xml#id(top.4)" role="scenario_topic_type"></nite:pointer>
<nite:child href="ES2002a.B.words.xml#id(ES2002a.B.words0)..id(ES2002a.B.words71)"></nite:child>
<nite:child href="ES2002a.D.words.xml#id(ES2002a.D.words0)..id(ES2002a.D.words3)"></nite:child>
<nite:child href="ES2002a.A.words.xml#id(ES2002a.A.words0)..id(ES2002a.A.words12)"></nite:child>
<nite:child href="ES2002a.B.words.xml#id(ES2002a.B.words72)..id(ES2002a.B.words73)"></nite:child>
<nite:child href="ES2002a.D.words.xml#id(ES2002a.D.words4)..id(ES2002a.D.words11)"></nite:child>
<nite:child href="ES2002a.C.words.xml#id(ES2002a.C.words0)..id(ES2002a.C.words7)"></nite:child>
<nite:child href="ES2002a.D.words.xml#id(ES2002a.D.words12)..id(ES2002a.D.words13)"></nite:child>
</topic>
<topic nite:id="ES2002a.topic.vka

The structure of the xml file after prasing can be found above:
```xml
<!-- Using for split subtopics -->
<topic>...</topic> 

<!-- Sub-subtopics -->
<nite:child href="ES2002a.B.words.xml#id(ES2002a.B.words0)..id(ES2002a.B.words71)"></nite:child>
```
- - -
information can be found, the subtopics should be split with `<topic>`, the sub-subtopics should have line break based on each `<nite:child>`

the information we need is the href which define three things:

`ES2002a.B.words`: the name of words' file should be located

`#id(ES2002a.B.words0)..id(ES2002a.B.words71)`: start word after `#id`, end word is `..id`

In [5]:
subtopic = xmlSoup1.find_all('topic') # find topic(subtopics)

In [6]:
subtopic[0]  # find sub-sub topics

<topic nite:id="ES2002a.topic.vkaraisk.1" other_description="introduction of participants and their roles">
<nite:pointer href="default-topics.xml#id(top.4)" role="scenario_topic_type"></nite:pointer>
<nite:child href="ES2002a.B.words.xml#id(ES2002a.B.words0)..id(ES2002a.B.words71)"></nite:child>
<nite:child href="ES2002a.D.words.xml#id(ES2002a.D.words0)..id(ES2002a.D.words3)"></nite:child>
<nite:child href="ES2002a.A.words.xml#id(ES2002a.A.words0)..id(ES2002a.A.words12)"></nite:child>
<nite:child href="ES2002a.B.words.xml#id(ES2002a.B.words72)..id(ES2002a.B.words73)"></nite:child>
<nite:child href="ES2002a.D.words.xml#id(ES2002a.D.words4)..id(ES2002a.D.words11)"></nite:child>
<nite:child href="ES2002a.C.words.xml#id(ES2002a.C.words0)..id(ES2002a.C.words7)"></nite:child>
<nite:child href="ES2002a.D.words.xml#id(ES2002a.D.words12)..id(ES2002a.D.words13)"></nite:child>
</topic>

In [7]:
type(subtopic[0])

bs4.element.Tag

In [8]:
sub_subtopic = subtopic[0].find_all('nite:child', href = True) # get information from href

In [9]:
sub_subtopic # list of sub-sub topics

[<nite:child href="ES2002a.B.words.xml#id(ES2002a.B.words0)..id(ES2002a.B.words71)"></nite:child>,
 <nite:child href="ES2002a.D.words.xml#id(ES2002a.D.words0)..id(ES2002a.D.words3)"></nite:child>,
 <nite:child href="ES2002a.A.words.xml#id(ES2002a.A.words0)..id(ES2002a.A.words12)"></nite:child>,
 <nite:child href="ES2002a.B.words.xml#id(ES2002a.B.words72)..id(ES2002a.B.words73)"></nite:child>,
 <nite:child href="ES2002a.D.words.xml#id(ES2002a.D.words4)..id(ES2002a.D.words11)"></nite:child>,
 <nite:child href="ES2002a.C.words.xml#id(ES2002a.C.words0)..id(ES2002a.C.words7)"></nite:child>,
 <nite:child href="ES2002a.D.words.xml#id(ES2002a.D.words12)..id(ES2002a.D.words13)"></nite:child>]

In [10]:
sub_subtopic[0].attrs # check information

{'href': 'ES2002a.B.words.xml#id(ES2002a.B.words0)..id(ES2002a.B.words71)'}

In [11]:
experssion = '(.*)\.words\.xml?(?:#id\(.*?\.words(\d+)\))(?:..id\(.*\.words(\d+)\))?'
re.findall(experssion,'ES2002a.B.words.xml#id(ES2002a.B.words0)..id(ES2002a.B.words71)') # match information using re

[('ES2002a.B', '0', '71')]

### Explore Segment file

In [12]:
xmlSoup2 = bsoup(open('./segments/ES2002a.A.segments.xml'),"lxml") # Extract information using lxml

The structure of the xml file after prasing can be found above:
```xml
<!-- Using for split segment -->
<segment>...</segment> 

<!-- segments range -->
<nite:child href="ES2002a.B.words.xml#id(ES2002a.B.words0)..id(ES2002a.B.words71)"></nite:child>
```
- - -
information can be found, the segments should be split with `<segment>`, there is one `<nite:child>` in each `<segment>`

the information we need is the href which define three things:

`ES2002a.B.words`: the name of words' file should be located

`#id(ES2002a.B.words0)..id(ES2002a.B.words71)`: start word after `#id`, end word is `..id`

In [13]:
segment = xmlSoup2.find_all('segment') # find segments

In [14]:
segment[0].find_all('nite:child', href = True) # get information in segments 

[<nite:child href="ES2002a.A.words.xml#id(ES2002a.A.words0)..id(ES2002a.A.words12)"></nite:child>]

In [15]:
child = segment[0].find_all('nite:child', href = True)

In [16]:
info = re.findall('.*?(?:#id\(.*?.words(\d+)\))(?:..id\(.*.words(\d+)\))?',child[0]['href'])[0]
info    # match information using re

('0', '12')

We can create a dictionary to store the segment information using python data structure for better efficiency,(access dictionary is much faster than file IO)
```python
{
    ('0','12'):'',
    ('12','20'):'',
    ...
}
```
- - -
Using word range as key, no need for value

### Explore Words file

In [17]:
xmlSoup3 = bsoup(open('./words/ES2002a.A.words.xml'),"lxml") # Extract information using lxml

The structure of the xml file after prasing can be found above:
```xml
<!-- Using for split words -->
<w>...</w> 
<w nite:id="ES2002a.A.words16" starttime="133.27" endtime="133.51">just</w>
```
- - -
Information can be found, the words have its `nite:id` can be used `number` for representing each word

the information we need is the id which define one things:

`ES2002a.A.words16`: This word number is 16, and word is `just`

In [18]:
xmlSoup3.find_all('w') # find words

[<w endtime="77.74" nite:id="ES2002a.A.words0" starttime="77.44">Hi</w>,
 <w endtime="77.74" nite:id="ES2002a.A.words1" punc="true" starttime="77.74">,</w>,
 <w endtime="78.16" nite:id="ES2002a.A.words2" starttime="77.74">I'm</w>,
 <w endtime="78.6" nite:id="ES2002a.A.words3" starttime="78.16">David</w>,
 <w endtime="78.85" nite:id="ES2002a.A.words4" starttime="78.6">and</w>,
 <w endtime="79.16" nite:id="ES2002a.A.words5" starttime="78.85">I'm</w>,
 <w endtime="79.53" nite:id="ES2002a.A.words6" starttime="79.16">supposed</w>,
 <w endtime="79.6" nite:id="ES2002a.A.words7" starttime="79.53">to</w>,
 <w endtime="79.74" nite:id="ES2002a.A.words8" starttime="79.6">be</w>,
 <w endtime="79.83" nite:id="ES2002a.A.words9" starttime="79.74">an</w>,
 <w endtime="80.21" nite:id="ES2002a.A.words10" starttime="79.83">industrial</w>,
 <w endtime="80.87" nite:id="ES2002a.A.words11" starttime="80.21">designer</w>,
 <w endtime="80.87" nite:id="ES2002a.A.words12" punc="true" starttime="80.87">.</w>,
 <w 

In [19]:
child = xmlSoup3.find_all('w')
wordinfo = int(re.findall('.*.words(\d+)',child[0]['nite:id'])[0]) # match word's number using re

In [20]:
wordinfo

0

We can create a dictionary to store the word information using python data structure for better efficiency,(access dictionary is much faster than file IO)
```python
{
    0:'Hi',
    1:',',
    2:"I'm",
    ...
}
```
- - -
Using word number as key, value is that word

## 4.Define the process functions, try batch processing

### Process words files

In [21]:
re.findall('(.*).words',xmlSoup3.find('nite:root')['nite:id'])[0] # using this as a name of the words file

'ES2002a.A'

In [22]:
def prase_words(file):
    '''
    input: xml file
    return: file's name and dictionary contain wrods,position pair
    '''
    
    xmlSoup = bsoup(file,"lxml")
    
    filename = re.findall('(.*).words',xmlSoup.find('nite:root')['nite:id'])[0]
    
    worddict = {}
    
    for words in xmlSoup.findAll('w'):
        wordnum = int(re.findall('.*[.]words(.*)',words['nite:id'])[0])
        worddict[wordnum] = words.string
    
    return filename, worddict
    

In [23]:
prase_words(open('./words/ES2002a.A.words.xml')) # test

('ES2002a.A',
 {0: 'Hi',
  1: ',',
  2: "I'm",
  3: 'David',
  4: 'and',
  5: "I'm",
  6: 'supposed',
  7: 'to',
  8: 'be',
  9: 'an',
  10: 'industrial',
  11: 'designer',
  12: '.',
  13: 'Um',
  14: ',',
  15: 'I',
  16: 'just',
  17: 'got',
  18: 'the',
  19: 'project',
  20: 'announcement',
  21: 'about',
  22: 'what',
  23: 'the',
  24: 'project',
  25: 'is',
  27: '.',
  28: 'Designing',
  29: 'a',
  30: 'remote',
  31: 'control',
  32: '.',
  33: "That's",
  34: 'about',
  35: 'it',
  36: ',',
  37: "didn't",
  38: 'get',
  39: 'anything',
  40: 'else',
  41: '.',
  42: 'Did',
  43: 'you',
  44: 'get',
  45: 'the',
  46: 'same',
  47: 'thing',
  48: '?',
  51: 'Cool',
  52: '.',
  53: "There's",
  54: 'too',
  55: 'much',
  56: 'gear',
  57: '.',
  60: 'Okay',
  61: '.',
  62: "Can't",
  63: 'draw',
  64: '.',
  66: 'Um',
  67: '.',
  68: 'Yeah',
  69: '.',
  70: 'Um',
  71: ',',
  72: 'well',
  73: 'anyway',
  74: ',',
  75: 'I',
  76: "don't",
  77: 'know',
  78: ',',
  79: "

### Process Segment files

In [24]:
re.findall('(.*).segs',xmlSoup2.find('nite:root')['nite:id'])[0]# using this as a name of the segment file

'ES2002a.A'

In [25]:
def prase_segments(file):
    '''
    input: xml file
    return: file's name and dictionary contain segment range
    '''    
    
    xmlSoup = bsoup(file,"lxml")
    
    filename = re.findall('(.*).segs',xmlSoup.find('nite:root')['nite:id'])[0]
    
    segmentdic = {}
    
    for segments in xmlSoup.findAll('segment'):
        for child in segments.findAll('nite:child', href = True):
            segmentRange = re.findall('.*?(?:#id\(.*?.words(\d+)\))(?:..id\(.*.words(\d+)\))?',child['href'])[0]
            segmentdic[segmentRange] = '' #store range as key, No value to save space
            
    
    return filename, segmentdic    

In [26]:
prase_segments(open('./segments/ES2002a.D.segments.xml')) # test

('ES2002a.D',
 {('0', '1'): '',
  ('2', '3'): '',
  ('4', '11'): '',
  ('12', '13'): '',
  ('14', '15'): '',
  ('16', '23'): '',
  ('24', '25'): '',
  ('26', '32'): '',
  ('33', '36'): '',
  ('37', '38'): '',
  ('39', '44'): '',
  ('45', '54'): '',
  ('55', '57'): '',
  ('58', ''): '',
  ('59', ''): '',
  ('60', '62'): '',
  ('63', '134'): '',
  ('135', ''): '',
  ('136', ''): '',
  ('137', '155'): '',
  ('156', '157'): '',
  ('158', ''): '',
  ('159', '161'): '',
  ('162', '163'): '',
  ('164', '170'): '',
  ('171', ''): '',
  ('172', '173'): '',
  ('174', '182'): '',
  ('183', ''): '',
  ('184', '189'): '',
  ('190', '194'): '',
  ('195', ''): '',
  ('196', ''): '',
  ('197', '209'): '',
  ('210', ''): '',
  ('211', '212'): '',
  ('213', '231'): '',
  ('232', ''): '',
  ('233', '234'): '',
  ('235', '244'): '',
  ('245', '256'): '',
  ('257', '282'): '',
  ('283', '293'): '',
  ('294', '295'): '',
  ('296', '297'): '',
  ('298', '299'): '',
  ('300', '302'): '',
  ('303', '304'): '',

## 5.Read words, Segments from files
- - -
### In this task, we should face 568 word files and 568 segment files, It may take a very long time for IO and processing, to deal with this situation, we can put them all togther into main memory(store in python data structure), using space exchange for time. The access for single element of dictionary have the time complexity $O(1)$, this method can give the code much faster running time

### Read words

We can create a dictionary to store the word information using python data structure for better efficiency,(access dictionary is much faster than file IO)
```python
{'ES2002a.A':{0:'Hi',1:',',2:"I'm",...},'ES2002a.B':{...,...},...}
```
- - -
Using word number as key, value is that word

In [27]:
# Batch process, read xml file as python data structurs (dict for better access)
worddict = {}
for xfile in os.listdir(words_file_path): 
    xfile = os.path.join(words_file_path, xfile)
    if os.path.isfile(xfile) and xfile.endswith('.xml'):
        fileName, words = prase_words(open(xfile))
        worddict[fileName] = words   

In [28]:
len(worddict)

568

In [29]:
worddict['ES2002a.A'] # Check result

{0: 'Hi',
 1: ',',
 2: "I'm",
 3: 'David',
 4: 'and',
 5: "I'm",
 6: 'supposed',
 7: 'to',
 8: 'be',
 9: 'an',
 10: 'industrial',
 11: 'designer',
 12: '.',
 13: 'Um',
 14: ',',
 15: 'I',
 16: 'just',
 17: 'got',
 18: 'the',
 19: 'project',
 20: 'announcement',
 21: 'about',
 22: 'what',
 23: 'the',
 24: 'project',
 25: 'is',
 27: '.',
 28: 'Designing',
 29: 'a',
 30: 'remote',
 31: 'control',
 32: '.',
 33: "That's",
 34: 'about',
 35: 'it',
 36: ',',
 37: "didn't",
 38: 'get',
 39: 'anything',
 40: 'else',
 41: '.',
 42: 'Did',
 43: 'you',
 44: 'get',
 45: 'the',
 46: 'same',
 47: 'thing',
 48: '?',
 51: 'Cool',
 52: '.',
 53: "There's",
 54: 'too',
 55: 'much',
 56: 'gear',
 57: '.',
 60: 'Okay',
 61: '.',
 62: "Can't",
 63: 'draw',
 64: '.',
 66: 'Um',
 67: '.',
 68: 'Yeah',
 69: '.',
 70: 'Um',
 71: ',',
 72: 'well',
 73: 'anyway',
 74: ',',
 75: 'I',
 76: "don't",
 77: 'know',
 78: ',',
 79: "it's",
 80: 'just',
 81: 'the',
 82: 'first',
 83: 'animal',
 84: 'I',
 85: 'can',
 86: 

### Read Segments

We can create a dictionary to store the segment information using python data structure for better efficiency,(access dictionary is much faster than file IO)
```python
{'ES2002a.A':{('0','12'):'',('12','20'):'',...},'ES2002a.B':{...,...},...}
```
- - -
Using word range as key, no need for value

In [30]:
# Batch process, read xml file as python data structurs (dict for better access)
segdict = {}
for xfile in os.listdir(segements_file_path): 
    xfile = os.path.join(segements_file_path, xfile)
    if os.path.isfile(xfile) and xfile.endswith('.xml'):
        fileName, segments = prase_segments(open(xfile))
        segdict[fileName] = segments

In [31]:
len(segdict)

568

In [32]:
segdict['ES2002a.A'] # Check result

{('0', '12'): '',
 ('13', '48'): '',
 ('49', ''): '',
 ('50', ''): '',
 ('51', '52'): '',
 ('53', '58'): '',
 ('59', ''): '',
 ('60', '61'): '',
 ('62', '65'): '',
 ('66', '67'): '',
 ('68', '69'): '',
 ('70', '120'): '',
 ('121', '143'): '',
 ('144', '152'): '',
 ('153', '162'): '',
 ('163', ''): '',
 ('164', ''): '',
 ('165', ''): '',
 ('166', '173'): '',
 ('174', ''): '',
 ('175', '176'): '',
 ('177', '203'): '',
 ('204', '207'): '',
 ('208', '209'): '',
 ('210', '211'): '',
 ('212', '217'): '',
 ('218', '253'): '',
 ('254', '256'): '',
 ('257', '258'): '',
 ('259', '260'): '',
 ('261', '269'): '',
 ('270', '290'): '',
 ('291', '298'): '',
 ('299', '301'): ''}

### Add the Segment to words(Breakdown with '\n')

Based on the specification, we need add the line break after each segments

In [33]:
# Add mark to words, using segment 
for i in segdict:
    for segrange in segdict[i]:
        if segrange[1] == '':  # if there is only one word in segment
            if int(segrange[0]) in worddict[i]:
                worddict[i][int(segrange[0])] += '\n' # Add segment in the word
            else:
                worddict[i][int(segrange[0])] = '\n' # No words in this position
        else: # if there is range of  words in segment
            if int(segrange[1]) in worddict[i]:
                worddict[i][int(segrange[1])] += '\n' # Add segment in the word
            else:
                worddict[i][int(segrange[1])] = '\n' # No words in this position 

In [34]:
worddict['ES2002a.A'] # Check result

{0: 'Hi',
 1: ',',
 2: "I'm",
 3: 'David',
 4: 'and',
 5: "I'm",
 6: 'supposed',
 7: 'to',
 8: 'be',
 9: 'an',
 10: 'industrial',
 11: 'designer',
 12: '.\n',
 13: 'Um',
 14: ',',
 15: 'I',
 16: 'just',
 17: 'got',
 18: 'the',
 19: 'project',
 20: 'announcement',
 21: 'about',
 22: 'what',
 23: 'the',
 24: 'project',
 25: 'is',
 27: '.',
 28: 'Designing',
 29: 'a',
 30: 'remote',
 31: 'control',
 32: '.',
 33: "That's",
 34: 'about',
 35: 'it',
 36: ',',
 37: "didn't",
 38: 'get',
 39: 'anything',
 40: 'else',
 41: '.',
 42: 'Did',
 43: 'you',
 44: 'get',
 45: 'the',
 46: 'same',
 47: 'thing',
 48: '?\n',
 51: 'Cool',
 52: '.\n',
 53: "There's",
 54: 'too',
 55: 'much',
 56: 'gear',
 57: '.',
 60: 'Okay',
 61: '.\n',
 62: "Can't",
 63: 'draw',
 64: '.',
 66: 'Um',
 67: '.\n',
 68: 'Yeah',
 69: '.\n',
 70: 'Um',
 71: ',',
 72: 'well',
 73: 'anyway',
 74: ',',
 75: 'I',
 76: "don't",
 77: 'know',
 78: ',',
 79: "it's",
 80: 'just',
 81: 'the',
 82: 'first',
 83: 'animal',
 84: 'I',
 85: 

## 6.Define the function process topic files

There are two different **Scenarios** for beginning of new lines: 

**Scenario1**
*	A linebreak after each <nite: child> tag of the topic files.
*	A linebreak after each segment.
*	10 '*' after each root topic (i.e. '\n**********\n').

**Scenario2**
* 10 '*' after each root topic (i.e. '\n**********\n')
* A linebreak after each segment

In this task, we using `Scenario1`, which we have linebreak for every segments, and also linebreak after sub-sub topics, if there is a conflict, we treat the sub-sub topics as a higher priority linebreak. 


In [35]:
def processTopicFile(file):
    '''
    input: topic xml files
    output: txt files
    '''
    
    xmlSoup = bsoup(file,"lxml")

    filename = re.findall('(.*).topic',xmlSoup.find('nite:root')['nite:id'])[0]
    
    topics = ''
    
    subtopics = xmlSoup.html.body.next.find_all('topic', recursive=False) # in case nested topics
    
    for subtopic in subtopics:
        child = subtopic.find_all(re.compile('nite:child')) # find only child elements
        topics += processSubTopic(child) # process topics line by line
        topics += '**********\n' # Add stars for each subtopics
        
    filename = './txt_files/'+ filename + '.txt' # process file's name
    
    output(topics,filename) # save to files


Define the function process subtopics 

In [36]:
def processSubTopic(subtopic):
    '''
    subtopic = subtopic.findAll('nite:child',href = True)
    split the subtopics line by line, add togther
    return: subtopic's text
    '''
    text = '' #initial subtopic's string
    
    for line in subtopic:
        info = re.findall('(.*)\.words\.xml?(?:#id\(.*?\.words(\d+)\))(?:..id\(.*\.words(\d+)\))?',line['href'])[0]
        filename = info[0] # match three information, file name, range of words
        wordrange = (info[1],info[2]) 
        if wordrange[1] != '': # if range exists
            start = int(wordrange[0])
            end = int(wordrange[1])
            for i in range(start,end+1):
                if i in worddict[filename]: # words have this index
                    if text == '':  # Nothing in the text
                        if worddict[filename][i] == '\n': # word is '\n'
                            continue
                        else:
                            text += ' '+ worddict[filename][i] # add this word
                    elif text[-1] == '\n' and (worddict[filename][i][0] == '\n'): # ensure no continue '\n'
                        continue 
                    else:
                        text += ' '+ worddict[filename][i] # add this word
                else:
                    continue # Not find word
        else:
            i = int(wordrange[0]) # Only one word in the range
            if i in worddict[filename]:
                if text == '':
                    if worddict[filename][i] == '\n':
                        continue
                    else:
                        text += ' '+ worddict[filename][i]
                elif worddict[filename][i][0] == '\n'and text[-1] == '\n':
                    continue    
                else:
                    text += ' '+ worddict[filename][i]
            else:
                continue
        if text != '': # add \n when there is no \n between subtopics
            if text[-1] != '\n':
                text += '\n'            
    return text


Define the output function, using text and filename as input, write a string to that file

In [37]:
def output(text,filename):
    '''
    input: Text (String),filename: get from file
    output: txt files
    '''
    text_file = open(filename, "w")
    text_file.write(text)
    text_file.close()

In [38]:
# Batch process
for xfile in os.listdir(topic_file_path): 
    xfile = os.path.join(topic_file_path, xfile)
    if os.path.isfile(xfile) and xfile.endswith('.xml'):
        processTopicFile(open(xfile))

## 7. References

1. [BeautifulSoup4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
2. [Python os](https://docs.python.org/3/library/os.html)
3. [Tutorial Material](https://moodle.vle.monash.edu/course/view.php?id=42906&section=17#17)
4. [Python data structure](https://docs.python.org/3/tutorial/datastructures.html)
5. [Python IO for files](https://docs.python.org/3/library/io.html?highlight=io#module-io)